In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install twilio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 23.6 MB/s 


## **Importing The ImageDataGenerator Library**

In [3]:
from keras.preprocessing.image import ImageDataGenerator

## **Define The Parameters /Arguments For ImageDataGenerator Class**

In [4]:
batch_size = 16

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# **Applying ImageDataGenerator Functionality To Trainset And Testset**

In [5]:
val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    r'/content/drive/Shareddrives/IBM - MAHALAKSHMI R/Final Deliverable/Dataset/train_set',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    r'/content/drive/Shareddrives/IBM - MAHALAKSHMI R/Final Deliverable/Dataset/test_set',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 436 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


## **Model Building**

## **Importing The Model Building Libraries**

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

## **Adding CNN & Dense Layers**

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


## **Configuring The Learning Process**

In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

## **Training and saving model**

In [10]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)
model.save('forest1.h5')

Epoch 1/10
28/28 [==============================] - 38s 1s/step - loss: 0.2126 - acc: 0.9266 - val_loss: 0.1714 - val_acc: 0.9256
Epoch 2/10
28/28 [==============================] - 33s 1s/step - loss: 0.2175 - acc: 0.9197 - val_loss: 0.0421 - val_acc: 0.9917
Epoch 3/10
28/28 [==============================] - 33s 1s/step - loss: 0.1832 - acc: 0.9358 - val_loss: 0.0194 - val_acc: 0.9917
Epoch 4/10
28/28 [==============================] - 32s 1s/step - loss: 0.2108 - acc: 0.9266 - val_loss: 0.0343 - val_acc: 0.9917
Epoch 5/10
28/28 [==============================] - 34s 1s/step - loss: 0.1752 - acc: 0.9289 - val_loss: 0.0379 - val_acc: 0.9917
Epoch 6/10
28/28 [==============================] - 36s 1s/step - loss: 0.1632 - acc: 0.9564 - val_loss: 0.0438 - val_acc: 0.9835
Epoch 7/10
28/28 [==============================] - 31s 1s/step - loss: 0.2039 - acc: 0.9289 - val_loss: 0.0376 - val_acc: 0.9835
Epoch 8/10
28/28 [==============================] - 34s 1s/step - loss: 0.2174 - acc: 0.94

## **Predictions**

In [15]:
import cv2
import numpy as np
from keras.utils import load_img, img_to_array
from keras.models import load_model
from twilio.rest import Client

model = load_model(r'forest1.h5')
name = ['forest', 'with fire']


img = load_img(r'/content/drive/Shareddrives/IBM - MAHALAKSHMI R/Final Deliverable/Dataset/test_set/with fire/19464620_401.jpg', target_size = (150,150))
x = img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)
p = pred[0][0].astype(int)
print(pred)
    

    
pred = model.predict(x)
if pred[0]==1:
    account_sid = 'AC03faa7b579ff4212e3a310a93e553e1a'
    auth_token = 'c6e2eaada8fdabad91c463189525cf37'
    client = Client(account_sid, auth_token)
    message = client.messages \
        .create(
            body = 'Forest Fire is detected, stay alert',
            from_='+12182559719',
            to='+919600909256'
        )
    print(message.sid)
    print('Fire Detected')
    print('SMS sent!')
        
else:
    print("No Danger")

1/1 [==============================] - 0s 123ms/step
[[1.]]
1/1 [==============================] - 0s 34ms/step
SMec0aa2c2147e8f323ffc81110c4b1b95
Fire Detected
SMS sent!
